In [ ]:
import json
import time
from datetime import datetime, timedelta
import random
import os

class MixingSimulation:
    def __init__(self):
        self.Volume = 200  # Total volume in Litres
        self.AM_ratio = 0.495
        self.PVDF_ratio = 0.05
        self.CB_ratio = 0.045
        self.NMP_ratio = 0.41

        # Density of each component (g/cm³)
        self.RHO_AM = 2.26
        self.RHO_CB = 1.8
        self.RHO_PVDF = 1.78
        self.RHO_NMP = 1

        # Empirical values of each component
        self.a = 0.85  # AM
        self.b = 2.2  # Binder
        self.c = 0.3  # CB
        self.s = -0.35 # Solvent

        # Initial component volumes
        self.NMP = self.NMP_ratio * self.Volume
        self.PVDF = 0
        self.CB = 0
        self.AM = 0

        self.results = []
        self.total_time = 0
        self.start_datetime = datetime.now()
        

    def run(self, step_percent=0.02, pause_sec=0.1):
        self._add_in_steps('PVDF', self.PVDF_ratio, step_percent, pause_sec, "PVDF")
        self._add_in_steps('CB', self.CB_ratio, step_percent, pause_sec, "Carbon Black")
        self._add_in_steps('AM', self.AM_ratio, step_percent, pause_sec, "Active Material")

    def _add_in_steps(self, component, ratio, step_percent, pause_sec, label):
        total_volume_to_add = ratio * self.Volume
        step_volume = step_percent * total_volume_to_add
        num_steps = int(1 / step_percent)

        last_saved_time = time.time()

        for _ in range(num_steps):
            self.total_time == 0
            setattr(self, component, getattr(self, component) + step_volume)

            timestamp = (self.start_datetime + timedelta(seconds=self.total_time)).isoformat()
            temperature = round(random.uniform(20, 25), 2)
            pressure = round(random.uniform(1, 2), 2)
            rpm = random.randint(300, 600)
            density = self._calculate_density()
            viscosity = self._calculate_viscosity()
            yield_stress = self._calculate_yield_stress()

            result = {
                "TimeStamp": timestamp,
                "Duration": self.total_time,
                "AM": round(self.AM, 3),
                "CB": round(self.CB, 3),
                "PVDF": round(self.PVDF, 3),
                "NMP": round(self.NMP, 3),
                "Temperature": temperature,
                "Pressure": pressure,
                "Speed (RPM)": rpm,
                "Density": round(density, 4),
                "Viscosity": round(viscosity, 2),
                "Yield Stress": round(yield_stress, 2)
            }
            self.results.append(result)

            now = time.time()
            if now - last_saved_time >= 0.1:
                os.makedirs("simulation_output", exist_ok=True)
                filename = f"simulation_output/result_at_{int(self.total_time)}s.json"
                try:
                    with open(filename, "w") as f:
                        json.dump(result, f, indent=4)
                except Exception as e:
                    print(f"Error writing result to file: {e}")
                last_saved_time = now
            self.total_time += 5
            time.sleep(pause_sec)

    def _calculate_density(self):
        m_NMP = self.NMP * self.RHO_NMP
        m_PVDF = self.PVDF * self.RHO_PVDF
        m_CB = self.CB * self.RHO_CB
        m_AM = self.AM * self.RHO_AM

        total_mass = m_NMP + m_PVDF + m_CB + m_AM
        total_volume = self.NMP + self.PVDF + self.CB + self.AM
        return total_mass / total_volume if total_volume > 0 else 0

    def _calculate_viscosity(self, max_solid_fraction=0.63, intrinsic_viscosity=3):
        total_volume = self.NMP + self.PVDF + self.CB + self.AM
        if total_volume == 0:
            return 0

        solid_volume = self.PVDF + self.CB + self.AM
        phi = solid_volume / total_volume
        if phi >= max_solid_fraction:
            phi = max_solid_fraction - 0.001

        try:
            viscosity = (1 - (phi / max_solid_fraction)) ** (-intrinsic_viscosity * max_solid_fraction)
            return viscosity * 2
        except Exception as e:
            print(f"Viscosity calculation error: {e}")
            return float('inf')

    def _calculate_yield_stress(self):
        m_NMP = self.NMP * self.RHO_NMP
        m_PVDF = self.PVDF * self.RHO_PVDF
        m_CB = self.CB * self.RHO_CB
        m_AM = self.AM * self.RHO_AM

        y_AM = self.a * m_AM
        y_PVDF = self.b * m_PVDF
        y_CB = self.c * m_CB
        y_NMP = self.s * m_NMP
        return y_NMP + y_PVDF + y_CB + y_AM

    def save_to_json(self, filename="simulation_output/final_results.json"):
        os.makedirs("simulation_output", exist_ok=True)
        try:
            with open(filename, "w") as f:
                json.dump(self.results, f, indent=4)
        except Exception as e:
            print(f"Failed to save final results: {e}")


if __name__ == "__main__":
    sim = MixingSimulation()
    sim.run()
    sim.save_to_json()



In [2]:
viscosity = sim.results[-1]["Viscosity"]
solidContent = sim.results[-1]["AM"] + sim.results[-1]["CB"] + sim.results[-1]["PVDF"]
density = sim.results[-1]["Density"]
print(solidContent)

118.0


In [ ]:
import numpy as np
import random
import time
from datetime import datetime, timedelta

class CoatingSimulation:
    def __init__(self):
        # Parameters (you can adjust them as necessary)
        self.coating_speed = 0.1  # m/s (example coating speed)
        self.gap_height = 50e-6  # Initial gap height in meters (example value)
        self.k = 0.5  # Viscosity constant (example value)
        self.n = 1.5  # Power-law index (example value)
        self.flow_rate = 0.01  # m³/s (example flow rate)
        self.coating_width = 0.5  # m (example coating width)
        self.solid_content = 60  # % (solid content in slurry)
        self.K = 2.5  # Defect risk constant (example value)
        self.base_std = 0.01  # Base standard deviation (example value)
        self.nominal_shear_rate = 50  # Example nominal shear rate (1/s)
        self.sample_points = 1000  # Number of samples for uniformity
        
        # Time settings
        self.start_time = datetime.now()
        self.total_time = 0  # in seconds

    def simulate(self, end_time=100):
        """Simulate the coating process and calculate parameters at each 5-second interval."""
        for t in range(0, end_time + 1, 5):  # Run every 5 seconds
            self.total_time = t
            shear_rate = self.calculate_shear_rate()
            viscosity = self.calculate_viscosity(shear_rate)
            wet_thickness = self.calculate_wet_thickness()
            dry_thickness = self.calculate_dry_thickness(wet_thickness)
            defect_risk = self.check_defect_risk(shear_rate, viscosity)
            uniformity_std = self.calculate_uniformity(shear_rate)

            # Simulate uniformity over time
            wet_thickness_array = wet_thickness + np.random.normal(0, uniformity_std, self.sample_points)

            # Print results for each time step
            print(f"At time {t}s:")
            print(f"  Shear Rate: {shear_rate:.4f} 1/s")
            print(f"  Viscosity: {viscosity:.4f} Pa.s")
            print(f"  Wet Thickness: {wet_thickness:.4f} m")
            print(f"  Dry Thickness: {dry_thickness:.4f} m")
            print(f"  Defect Risk: {defect_risk}")
            print(f"  Uniformity STD: {uniformity_std:.4f}")
            print("  ------------------------")
            
            # Simulate time delay (optional, for realistic simulation)
            time.sleep(0.1)

    def calculate_shear_rate(self):
        """Calculate the shear rate based on coating speed and gap height."""
        return self.coating_speed / self.gap_height

    def calculate_viscosity(self, shear_rate):
        """Calculate viscosity using power-law fluid model."""
        return self.k * (shear_rate ** (self.n - 1))

    def calculate_wet_thickness(self):
        """Calculate the wet thickness of the coating."""
        return self.flow_rate / (self.coating_speed * self.coating_width)

    def calculate_dry_thickness(self, wet_thickness):
        """Calculate the dry thickness of the coating based on solid content."""
        return wet_thickness * (self.solid_content / 100)

    def check_defect_risk(self, shear_rate, viscosity):
        """Check if defect risk is high based on coating speed and gap height."""
        if (self.coating_speed / self.gap_height) > (self.K * viscosity):
            return True
        else:
            return False

    def calculate_uniformity(self, shear_rate):
        """Calculate uniformity standard deviation based on shear rate."""
        return self.base_std * (shear_rate / self.nominal_shear_rate)

# Run the simulation
if __name__ == "__main__":
    sim = CoatingSimulation()
    sim.simulate(end_time=100)  # Simulate for 100 seconds with 5-second intervals


At time 0s:
  Shear Rate: 2000.0000 1/s
  Viscosity: 22.3607 Pa.s
  Wet Thickness: 0.2000 m
  Dry Thickness: 0.1200 m
  Defect Risk: True
  Uniformity STD: 0.4000
  ------------------------
At time 5s:
  Shear Rate: 2000.0000 1/s
  Viscosity: 22.3607 Pa.s
  Wet Thickness: 0.2000 m
  Dry Thickness: 0.1200 m
  Defect Risk: True
  Uniformity STD: 0.4000
  ------------------------
At time 10s:
  Shear Rate: 2000.0000 1/s
  Viscosity: 22.3607 Pa.s
  Wet Thickness: 0.2000 m
  Dry Thickness: 0.1200 m
  Defect Risk: True
  Uniformity STD: 0.4000
  ------------------------
At time 15s:
  Shear Rate: 2000.0000 1/s
  Viscosity: 22.3607 Pa.s
  Wet Thickness: 0.2000 m
  Dry Thickness: 0.1200 m
  Defect Risk: True
  Uniformity STD: 0.4000
  ------------------------
At time 20s:
  Shear Rate: 2000.0000 1/s
  Viscosity: 22.3607 Pa.s
  Wet Thickness: 0.2000 m
  Dry Thickness: 0.1200 m
  Defect Risk: True
  Uniformity STD: 0.4000
  ------------------------
At time 25s:
  Shear Rate: 2000.0000 1/s
  Visc

: 